# SCADA Flow Rate Calculation Model

In [1]:
import pandas as pd
import os
import csv
import math

import re
import datetime
import io
from zipfile import ZipFile

import pandas as pd
import numpy as np
import pandas_gbq
import janitor

# Section 1

In this section, I read my Lift Station SCADA data and Lift Station Legend data to a pandas data frames and do some quick formatting. I'll begin by converting the date and time column to DateTime format, dropping unneeded columns, and dropping any NA values. Once this process is finished we are ready to begin section 2.

## Read SCADA data into Pandas Dataframe

In [2]:
df = pd.read_csv('LiftStation.csv')

/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_11332/2210235391.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('LiftStation.csv')


In [3]:
legend = pd.read_csv('Lift Station Legend Cross Referance.csv')

In [4]:
legend = legend.drop('Pump #1 Run', axis=1)
legend = legend.drop('Pump #2 Run', axis=1)
legend = legend.drop('Pump #3 Run', axis=1)
legend = legend.drop('Table Tag #.4', axis=1)
legend = legend.drop('Pump #4 Run', axis=1)
legend = legend.drop('Unnamed: 11', axis=1)

## Convert the DateAndTime column to Datetime Format

In [5]:
df['PumpStart'] = pd.to_datetime(df['PumpStart'])
df['PumpStop'] = pd.to_datetime(df['PumpStop'])

## Format the dataframe

In [6]:
df = df.dropna(subset=['PumpStart', 'PumpStop'])

In [7]:
df['PumpTagIndex'] = df['PumpTagIndex'].astype(int)

In [8]:
# create a boolean mask of rows with duplicate "PumpTagIndex" and "PumpStart" values
mask = df.duplicated(subset =['PumpTagIndex', 'PumpStart'], keep = 'first')

# drop the duplicate rows using the boolean mask

#now let's drop the duplicate rows
new_df = df[~mask]
# reset the index after dropping rows
new_df.reset_index(drop=True, inplace=True)

# print the length of the new dataframe
#print(len(new_df))

In [9]:
# create a boolean mask of rows with duplicate "PumpTagIndex" and "PumpStart" values
mask = new_df.duplicated(subset=['PumpTagIndex', 'PumpStop'], keep='first')

# drop the duplicate rows using the boolean mask
new_df = new_df[~mask]

# reset the index after dropping rows
new_df.reset_index(drop=True, inplace=True)

# print the length of the new dataframe
#print(len(new_df))

# Section 2

In this section, I will create a new data frame called calc_df, which will hold all of our calculated data going forward. 

I'll begin by assigning the original df to my calculated df, and reformatting the columns (names and order). 

Next I will remove any erroneous rows containing invalid pump numbers.

Then I'll use the legend dataframe I read into the program earlier to add the correct lift station name to each row in calc_df.

Finally, I will convert both on and off timestamps into Unix time and calculate the difference in time between the pump turning on and off. 

Note: Remove the # in front of any of the calc_df lines to see a preview of the data frame.

## Create new data frame

In [10]:
calc_df = new_df
calc_df = calc_df.rename(columns={'PumpStart': 'PumpStartTime'})
calc_df = calc_df.rename(columns={'PumpStop': 'PumpStopTime'})
calc_df = calc_df.rename(columns={'WellStart': 'StartWellLevel'})
calc_df = calc_df.rename(columns={'WellEnd': 'StopWellLevel'})

In [11]:
calc_df['LiftStation'] = None
calc_df['StartUnix'] = 0.0
calc_df['StopUnix'] = 0.0
calc_df['Time_diff'] = 0.0

In [12]:
calc_df = calc_df[['LiftStation','PumpTagIndex', 'PumpStartTime', 'StartWellLevel','PumpStopTime', 'StopWellLevel', 'StartUnix', 'StopUnix','Time_diff']]

In [13]:
#calc_df

## Delete any rows with incorrect pump numbers

In [14]:
#create a list of valid pump numbers

valid_pumps = []

for index, row in legend.iterrows():
    valid_pumps.append(row['Table Tag #'])
    valid_pumps.append(row['Table Tag #.1'])
    valid_pumps.append(row['Table Tag #.2'])
    valid_pumps.append(row['Table Tag #.3'])
    
valid_pumps = list(filter(lambda x: not math.isnan(x), valid_pumps))
        
#len(valid_pumps)

In [15]:
calc_df = calc_df[calc_df['PumpTagIndex'].isin(valid_pumps)]
calc_df = calc_df.reset_index(drop=True)
calc_df['PumpTagIndex'] = calc_df['PumpTagIndex'].astype(int)

## Add the correct Lift Station to each row in calc_df

Here I am going to perfom 4 joins between the legend dataframe and my calc_df. Each join will merge the legend and calc data frames on one of the 4 table tag #'s.

Table Tag 0 Join

In [16]:
# Left join calc_df to legend df where Pump Tag Index = Table Tag
merged_df = calc_df.merge(legend, left_on='PumpTagIndex', right_on='Table Tag #', how='left')

# Create a mask to filter the rows where Lift Station is not empty
mask = merged_df['Lift Station'].notnull()

# Assign the Lift Station values to the LiftStation column in the calc_df dataframe
calc_df.loc[mask, 'LiftStation'] = merged_df.loc[mask, 'Lift Station']

Table Tag 1 Join

In [17]:
merged_df = calc_df.merge(legend, left_on='PumpTagIndex', right_on='Table Tag #.1', how='left')

# Create a mask to filter the rows where Lift Station is not empty
mask = merged_df['Lift Station'].notnull()

# Assign the Lift Station values to the LiftStation column in the calc_df dataframe
calc_df.loc[mask, 'LiftStation'] = merged_df.loc[mask, 'Lift Station']

Table Tag 2 Join

In [18]:
merged_df = calc_df.merge(legend, left_on='PumpTagIndex', right_on='Table Tag #.2', how='left')

# Create a mask to filter the rows where Lift Station is not empty
mask = merged_df['Lift Station'].notnull()

# Assign the Lift Station values to the LiftStation column in the calc_df dataframe
calc_df.loc[mask, 'LiftStation'] = merged_df.loc[mask, 'Lift Station']

Table Tag 3 Join

In [19]:
merged_df = calc_df.merge(legend, left_on='PumpTagIndex', right_on='Table Tag #.3', how='left')

# Create a mask to filter the rows where Lift Station is not empty
mask = merged_df['Lift Station'].notnull()

# Assign the Lift Station values to the LiftStation column in the calc_df dataframe
calc_df.loc[mask, 'LiftStation'] = merged_df.loc[mask, 'Lift Station']

## Now let's get Unix time for each timestamp

In [20]:
calc_df['StartUnix'] = pd.to_datetime(calc_df['PumpStartTime']).astype(int)/ 10**9
calc_df['StopUnix'] = pd.to_datetime(calc_df['PumpStopTime']).astype(int) / 10**9

In [21]:
calc_df.head(20)

,LiftStation,PumpTagIndex,PumpStartTime,StartWellLevel,PumpStopTime,StopWellLevel,StartUnix,StopUnix,Time_diff
0,Momont One (15),871,2022-06-24 00:01:03,4.017924,2022-06-24 00:02:23,1.005562,1.656029e+09,1.656029e+09,0.0
1,Momont Two (18),896,2022-06-24 00:01:58,7.711633,2022-06-24 00:07:01,3.349951,1.656029e+09,1.656029e+09,0.0
2,Caras Park (7),832,2022-06-24 00:03:01,7.636364,2022-06-24 00:03:57,4.703812,1.656029e+09,1.656029e+09,0.0
3,Linda Vista Golf Course (12),774,2022-06-24 00:03:53,5.805737,2022-06-24 00:05:37,2.844790,1.656029e+09,1.656029e+09,0.0
4,East Broadway (9),682,2022-06-24 00:04:59,6.077348,2022-06-24 00:06:29,3.852914,1.656029e+09,1.656029e+09,0.0
5,University (8),666,2022-06-24 00:05:25,3.450440,2022-06-24 00:06:31,1.953079,1.656029e+09,1.656029e+09,0.0
6,Dickens Street (17),880,2022-06-24 00:05:59,3.424242,2022-06-24 00:06:38,2.466276,1.656029e+09,1.656029e+09,0.0
7,Community Hospital (6),651,2022-06-24 00:07:05,2.451613,2022-06-24 00:12:01,1.919941,1.656029e+09,1.656030e+09,0.0
8,Caras Park (7),831,2022-06-24 00:08:06,7.448680,2022-06-24 00:09:08,4.797654,1.656029e+09,1.656029e+09,0.0
9,Dickens Street (17),879,2022-06-24 00:09:06,3.385142,2022-06-24 00:09:43,2.329423,1.656029e+09,1.656029e+09,0.0


## Calculating the time difference between Pumps turning on and off

In [22]:
calc_df['Time_diff'] = (calc_df['StopUnix'] - calc_df['StartUnix']) / 60

# Section 3

In this section, I will add elevation difference, volume per foot, observed pumping flow rate, the average flow rate per pump, and average observed pumping flow rate (GPM).

Elevation Difference - This value can be changed for the desired lift station. Please note that each lift station will have a different elevation level. To change the elevation for a different lift station, simply change the 2.2 in the first cell below.

Observed pumping flow rate - To calculate this value we first need to input the wet wells diameter, again this value will differ per each lift station. Simply change the diameter in the diam_well = (5) statement to the desired value. From there the vol_per_foot calculation will update and be added to the calc_df. Now we have all the information in the calc_df to calculate the observed pumping flow rate. I'll take the elevation difference value times the volume per foot value, and divide the result by our time difference. The resulting value will be added to the calc_df in the observed pumping flow rate column.

Average flow rate per pump - Average flow rate per pump is calculated by adding the values in the observed pumping flow rate column for each pump and then dividing that sum by the count of values for that pump. So if I iterate over 4 rows of the calc_df, and pump 1 has 3 out of the 4 rows with an observed pumping flow rate of 100, 120, and 100, the formula will sum these three rows (320) and divide by the number of rows that were related to pump 1 (3). The average flow rate per pump 1 in row 4 would be 106.

Average observed pumping flow rate (GPM) - to calculate this value I simply add all the average flow rates for both pumps and divide the sum by the count of rows I have iterated over. For example, if I have iterated over 4 rows, and 2 of the rows are pump 1 and 2 are pump 2, I'll add up the values from both pumps (100,110,100,120) and divide the sum by 4 to calculate the average observed pumping flow rate in GPM of 107.5 in row 4.



## Add in Elevation Difference

In [23]:
calc_df['Elevation_dif'] = 0.0

In [24]:
legend['Elevation_diff'] = 2.2

In [25]:
legend.loc[0, 'Elevation_diff'] = 3.3

In [26]:
#merge legend elevation_diff to calc_df

merged_df = calc_df.merge(legend, left_on = 'LiftStation', right_on = 'Lift Station', how = 'left')

mask = merged_df['Elevation_diff'].notnull()

calc_df.loc[mask, 'Elevation_dif'] = merged_df.loc[mask, 'Elevation_diff']

## Calculate Observed Pumping Flow Rate

### Calculate Volume Per Foot for each well

In [27]:
#initialize new column
calc_df['wet_well_diam'] = 0.0

# left join calc_df and legend df on lift station
merged_df = calc_df.merge(legend, left_on = 'LiftStation', right_on = 'Lift Station', how = 'left')

#create a boolean mask to find all rows where we have a wet well diameter
mask = merged_df['Wet Well Diam'].notnull()

#add the wet well diameter value from the legend dataframe to the calc_df where mask is true
calc_df.loc[mask, 'wet_well_diam'] = merged_df.loc[mask, 'Wet Well Diam']

In [28]:
#Calculate volume per foot (check 4*7.48)
calc_df['volume_per_foot'] = round(((calc_df['wet_well_diam']*calc_df['wet_well_diam']) * 3.14)/4*7.48)

### Calculate Observed Pumping Flow Rate

In [29]:
calc_df['observed_pumping_flow_rate'] = int()

In [30]:
for index, row in calc_df.iterrows():
    result = round((row['Elevation_dif'] * row['volume_per_foot'])/row['Time_diff'])
    calc_df.at[index,'observed_pumping_flow_rate'] = result

In [31]:
#calc_df

## Calculate Average Flow Rate Per Pump

In [32]:
calc_df['avg_flow_rate_pump'] = int()

### Create two dictionaries
### 1. Dictionary containing a running total of Observed Pumping Flow Rate for each pump
### 2. Dictionary containing a running row count of Observed Pumping Flow Rate for each pump

In [33]:
sum_dict ={}
count_dict ={}

for item in valid_pumps:
    result = 0
    sum_dict[item] = result
    count_dict[item] = result

### Calculate Average Flow Rate Per Pump

In [34]:
for index, row in calc_df.iterrows():
    tag = row["PumpTagIndex"]
    sum_dict[tag] += row['observed_pumping_flow_rate']
    count_dict[tag] += 1
    calc_df.at[index, 'avg_flow_rate_pump'] = sum_dict[tag] / count_dict[tag]    

## Finally Let's calculate our Average Observed Pumping Flow Rate (GPM)

In [35]:
calc_df['average_observed_pumping_flow_rate'] = int()

### Create a dictionary for sum and count for each Lift Station

In [36]:
sum_dict ={}
count_dict ={}

for index, row in legend.iterrows():
    result = 0
    tag = row['Lift Station']
    sum_dict[tag] = result
    count_dict[tag] = result

### Calculate Average Observed Pumping Flow Rate (GPM)

In [37]:
for index, row in calc_df.iterrows():
    tag = row["LiftStation"]
    sum_dict[tag] += row['observed_pumping_flow_rate']
    count_dict[tag] += 1
    calc_df.at[index, 'average_observed_pumping_flow_rate'] = sum_dict[tag] / count_dict[tag]  

In [38]:
#filtered_df = calc_df[calc_df['LiftStation'] == 'Waldo (22)']
#filtered_df.head(60)

# Section 4

In this section, I will calculate the time to fill, inflow, and average inflow. These values need to be accounted for when calculating flow rate, as even when the pumps are turned on within a wet well and the level is dropping, water is still flowing through the sewer pipes into the well.

Time to fill - To calculate the time to fill for each row in calc_df, I take the stop Unix time of the first row within calc_df (when the pump turned off) and subtract this value from the start Unix time of the next row (when the pump turned back on). The difference between the pump turning off and the next pump turning on is my time to fill the measure.

Inflow - To calculate my inflow value for each row in calc_df, I take the row's volume per foot value and multiply it by the row's elevation difference value. I then divide that result by the row's time to fill value we just calculated.

Average Inflow - To calculate my average inflow for each row in calc_df, I simply add all the values for inflow in the rows I have iterated over and divide by the count of those rows. So if I have iterated over 4 rows so far in calc_df and the inflow values are (60,40,20,60) I sum these values (180) and divide by four to get my average inflow value in row 4, 45.


## Calculate time to fill

In [39]:
calc_df['Time_to_Fill'] = float

### Create two dictionaries
### 1. Dictionary containing the Start Unix time for each Lift Station
### 2. Dictionary containing the Stop Unix time for each Lift Station

In [40]:
# Sort the dataframe by LiftStation and PumpStartTime
calc_df = calc_df.sort_values(by=['LiftStation', 'PumpStartTime'])
calc_df = calc_df.reset_index(drop=True)

In [53]:

for index, row in calc_df.iterrows():
    if index == 0:
        calc_df.at[index, 'Time_to_Fill'] = 0
    
    if row['LiftStation'] == calc_df.iloc[index-1]['LiftStation']:
        milliseconds = (row['StartUnix'] - calc_df.iloc[index-1]['StopUnix']) 
        seconds = milliseconds / 10
        minutes = seconds // 60
        seconds = seconds % 60
        result = minutes + seconds / 60
        calc_df.at[index, 'Time_to_Fill'] = result
        print(result)
        
    if row['LiftStation'] != calc_df.iloc[index-1]['LiftStation']:
        result = 0
        calc_df.at[index, 'Time_to_Fill'] = result
 
        

19.998333333333335
16.885
6.633333333333333
7.4383333333333335
7.286666666666666
8.718333333333334
9.171666666666665
11.108333333333333
5.86
6.375
7.17
7.623333333333333
6.261666666666667
5.751666666666667
7.085
8.196666666666667
8.648333333333333
13.985000000000001
11.686666666666667
10.01
6.37
4.198333333333333
5.664999999999999
5.598333333333333
4.585000000000001
5.163333333333333
7.551666666666667
6.298333333333333
3.706666666666667
2.4050000000000002
2.61
2.3616666666666664
2.5716666666666668
3.2383333333333337
6.866666666666667
4.305000000000001
3.6
4.5216666666666665
7.614999999999999
6.366666666666666
15.49
13.6
12.51
8.275
2.8583333333333334
4.0616666666666665
5.566666666666666
4.611666666666666
7.696666666666667
6.239999999999999
6.841666666666667
6.036666666666666
8.405
5.175
3.96
6.543333333333334
4.221666666666667
3.04
4.131666666666667
6.371666666666667
13.543333333333333
14.776666666666667
8.381666666666666
8.886666666666667
5.911666666666666
4.515
2.996666666666667
3.77

7.59
5.83
4.961666666666666
7.423333333333333
6.951666666666667
6.191666666666666
6.071666666666667
5.601666666666667
4.045
3.195
3.0949999999999998
3.5833333333333335
3.59
4.539999999999999
10.106666666666666
11.985000000000001
10.226666666666667
7.473333333333333
4.735
5.180000000000001
7.7283333333333335
5.258333333333334
4.995
5.296666666666667
4.988333333333333
5.838333333333334
4.461666666666667
7.4
5.986666666666666
7.703333333333333
5.453333333333333
3.615
4.816666666666666
5.36
5.475
8.901666666666667
7.968333333333334
11.216666666666667
9.176666666666668
9.18
4.288333333333333
5.02
4.661666666666666
5.425
6.39
3.1
4.803333333333333
3.8216666666666668
6.41
4.178333333333333
5.38
7.526666666666667
5.608333333333333
3.9983333333333335
4.081666666666667
4.7
4.755
4.383333333333334
3.668333333333333
4.6
3.9800000000000004
4.4783333333333335
19.285
9.583333333333334
6.205
6.951666666666667
7.4
4.3100000000000005
4.645
4.265
7.89
3.703333333333333
4.510000000000001
6.083333333333333

8.845
3.7816666666666667
2.4116666666666666
3.731666666666667
5.61
4.053333333333333
5.091666666666667
3.3116666666666665
4.725
2.4800000000000004
3.7383333333333333
6.238333333333333
6.158333333333333
6.166666666666667
6.648333333333333
5.205
5.498333333333333
5.793333333333334
4.363333333333333
5.739999999999999
8.513333333333334
5.1066666666666665
7.386666666666667
9.268333333333334
7.2733333333333325
6.095
6.656666666666666
4.963333333333334
2.871666666666667
5.558333333333334
5.493333333333334
4.8133333333333335
4.933333333333334
5.153333333333333
4.651666666666667
3.35
4.586666666666666
4.261666666666667
3.3083333333333336
4.885000000000001
3.39
5.8100000000000005
4.19
3.2266666666666666
6.1883333333333335
7.941666666666666
6.698333333333333
12.946666666666665
11.096666666666666
8.786666666666667
4.04
2.5
5.34
5.7683333333333335
9.526666666666667
5.775
7.555
10.746666666666666
7.748333333333333
4.735
5.545
6.316666666666666
3.4833333333333334
4.2316666666666665
5.78
4.485
5.50333

6.89
3.915
4.153333333333333
4.793333333333334
7.1033333333333335
9.195
16.351666666666667
5.851666666666667
12.796666666666665
4.243333333333333
4.558333333333334
4.848333333333333
6.4766666666666675
7.121666666666667
5.006666666666666
4.213333333333334
0.06666666666666667
7.7283333333333335
5.013333333333334
5.083333333333333
3.533333333333333
4.148333333333333
6.67
3.731666666666667
3.545
4.941666666666666
6.989999999999999
6.191666666666666
6.7716666666666665
7.178333333333333
10.661666666666667
12.908333333333333
9.761666666666667
6.6466666666666665
2.9716666666666667
3.6333333333333333
3.6
3.825
6.2716666666666665
4.648333333333333
5.905
2.8583333333333334
3.2783333333333333
3.565
4.371666666666667
6.011666666666667
5.7683333333333335
3.5616666666666665
3.65
5.616666666666667
2.96
2.8333333333333335
4.903333333333333
5.1850000000000005
6.776666666666667
13.455
10.673333333333334
9.466666666666667
10.416666666666666
5.826666666666667
1.955
2.191666666666667
2.268333333333333
2.023

4.888333333333334
4.625
4.096666666666667
3.6016666666666666
3.5199999999999996
4.945
8.063333333333334
9.408333333333333
14.833333333333334
14.595
9.56
4.108333333333333
4.3133333333333335
3.7916666666666665
5.121666666666667
7.535
8.553333333333335
6.381666666666666
5.883333333333333
5.795
4.180000000000001
3.74
4.001666666666667
3.515
3.4333333333333336
3.5300000000000002
3.3616666666666664
5.66
7.455
9.698333333333332
6.3149999999999995
18.40666666666667
8.733333333333333
4.875
4.093333333333333
7.681666666666667
8.041666666666666
6.706666666666666
7.451666666666667
6.364999999999999
8.741666666666667
6.93
5.53
5.045
6.78
3.975
6.141666666666667
10.575
8.615
8.105
12.143333333333334
9.46
5.766666666666667
4.6866666666666665
4.391666666666667
4.8533333333333335
5.358333333333333
5.489999999999999
5.241666666666666
7.583333333333333
6.6066666666666665
7.9816666666666665
5.611666666666666
5.948333333333333
3.6733333333333333
4.1883333333333335
2.0783333333333336
4.968333333333334
2.69

2.9699999999999998
4.53
3.065
4.053333333333333
4.116666666666666
2.555
4.093333333333333
4.068333333333333
3.7699999999999996
3.418333333333333
3.5199999999999996
3.5983333333333336
3.265
4.6466666666666665
4.173333333333334
2.503333333333333
3.6133333333333333
3.6666666666666665
2.953333333333333
3.7449999999999997
2.831666666666667
3.2616666666666667
4.496666666666667
3.9016666666666664
4.083333333333333
5.876666666666667
5.556666666666667
5.016666666666667
4.658333333333333
5.538333333333334
5.193333333333333
3.6916666666666664
3.7816666666666667
3.94
3.8666666666666667
3.2783333333333333
3.3
2.9633333333333334
3.0900000000000003
0.6566666666666666
0.021666666666666667
4.843333333333334
2.8949999999999996
5.095
4.178333333333333
4.381666666666666
3.3333333333333335
4.008333333333334
3.5883333333333334
3.5300000000000002
2.5083333333333333
2.3666666666666667
3.82
2.3616666666666664
2.8433333333333333
3.091666666666667
2.8833333333333333
3.7449999999999997
4.526666666666667
4.0550000

3.975
3.8550000000000004
3.4699999999999998
4.1866666666666665
4.710000000000001
4.0616666666666665
3.1266666666666665
2.7800000000000002
3.066666666666667
3.021666666666667
3.7866666666666666
2.4466666666666668
2.3233333333333333
2.796666666666667
3.121666666666667
3.01
3.935
3.9383333333333335
4.578333333333333
4.633333333333333
4.3
9.45
4.786666666666666
4.256666666666667
3.0533333333333332
3.371666666666667
4.031666666666667
3.631666666666667
3.091666666666667
2.993333333333333
2.665
3.7816666666666667
3.4016666666666664
1.9366666666666668
3.5316666666666667
3.705
3.253333333333333
2.671666666666667
3.82
3.728333333333333
4.166666666666667
3.6133333333333333
3.643333333333333
3.3916666666666666
2.6616666666666666
3.445
4.245
2.966666666666667
2.8766666666666665
2.545
2.543333333333333
3.0383333333333336
2.71
3.6416666666666666
3.1733333333333333
2.9066666666666667
2.8266666666666667
4.538333333333334
4.586666666666666
4.621666666666667
4.2733333333333325
4.505
4.55
3.85666666666666

2.01
3.9733333333333336
4.3149999999999995
2.825
3.3866666666666667
3.1666666666666665
2.805
4.691666666666666
2.8916666666666666
3.805
4.628333333333333
4.963333333333334
4.905
5.446666666666667
5.46
4.906666666666666
4.645
5.348333333333333
3.0633333333333335
3.4766666666666666
3.2199999999999998
2.5766666666666667
2.746666666666667
2.8516666666666666
2.8583333333333334
3.5716666666666668
2.25
3.96
2.203333333333333
0.015000000000000001
3.41
4.155
4.743333333333334
4.636666666666667
3.56
3.1383333333333336
4.46
3.396666666666667
4.023333333333333
2.32
2.0466666666666664
1.7383333333333333
3.2216666666666667
3.9316666666666666
3.6816666666666666
3.296666666666667
3.44
3.2866666666666666
4.113333333333333
4.35
4.385000000000001
10.338333333333333
5.681666666666667
4.325
2.5050000000000003
4.133333333333334
3.1666666666666665
3.6533333333333333
2.84
3.21
3.7666666666666666
2.9433333333333334
2.993333333333333
3.2733333333333334
4.180000000000001
3.3833333333333333
2.9016666666666664
4.0

KeyboardInterrupt: 

In [54]:
calc_df

,LiftStation,PumpTagIndex,PumpStartTime,StartWellLevel,PumpStopTime,StopWellLevel,StartUnix,StopUnix,Time_diff,Elevation_dif,wet_well_diam,volume_per_foot,observed_pumping_flow_rate,avg_flow_rate_pump,average_observed_pumping_flow_rate,Time_to_Fill
0,Buena Vista (19),920,2022-06-24 00:11:33,3.200578,2022-06-24 00:13:32,2.189985,1.656029e+09,1.656030e+09,1.983333,3.3,5,147.0,245,245.000000,245.000000,0
1,Buena Vista (19),918,2022-06-24 03:33:31,3.196136,2022-06-24 03:35:31,2.187765,1.656042e+09,1.656042e+09,2.000000,3.3,5,147.0,243,243.000000,244.000000,19.998333
2,Buena Vista (19),920,2022-06-24 06:24:22,3.198357,2022-06-24 06:26:18,2.232186,1.656052e+09,1.656052e+09,1.933333,3.3,5,147.0,251,248.000000,246.333333,16.885
3,Buena Vista (19),918,2022-06-24 07:32:38,3.200578,2022-06-24 07:34:38,2.229965,1.656056e+09,1.656056e+09,2.000000,3.3,5,147.0,243,243.000000,245.500000,6.633333
4,Buena Vista (19),920,2022-06-24 08:49:01,3.196136,2022-06-24 08:50:59,2.192206,1.656061e+09,1.656061e+09,1.966667,3.3,5,147.0,247,247.666667,245.800000,7.438333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383400,Waldo (22),1033,2023-01-29 23:14:11,4.922175,2023-01-29 23:15:17,3.814686,1.675034e+09,1.675034e+09,1.100000,2.2,8,376.0,752,810.780936,836.550676,58.866667
383401,Waldo (22),1032,2023-01-30 08:24:31,4.919978,2023-01-30 08:25:38,3.812489,1.675067e+09,1.675067e+09,1.116667,2.2,8,376.0,741,862.640545,836.470042,549.233333
383402,Waldo (22),1033,2023-01-30 11:15:06,5.019593,2023-01-30 11:16:12,3.816151,1.675077e+09,1.675077e+09,1.100000,2.2,8,376.0,752,810.682805,836.398820,169.466667
383403,Waldo (22),1032,2023-01-30 17:08:39,4.926570,2023-01-30 17:09:47,3.818348,1.675099e+09,1.675099e+09,1.133333,2.2,8,376.0,730,862.414966,836.309183,352.45


In [49]:
start = calc_df['StartUnix'].tolist()
stop = calc_df['StopUnix'].tolist()

In [51]:
stop

[1656029612.0,
 1656041731.0,
 1656051978.0,
 1656056078.0,
 1656060659.0,
 1656065156.0,
 1656070504.0,
 1656076132.0,
 1656082915.0,
 1656086557.0,
 1656090497.0,
 1656094921.0,
 1656099611.0,
 1656103491.0,
 1656107062.0,
 1656111438.0,
 1656116473.0,
 1656121782.0,
 1656130287.0,
 1656137420.0,
 1656143540.0,
 1656147488.0,
 1656150126.0,
 1656153648.0,
 1656157128.0,
 1656160001.0,
 1656163222.0,
 1656167876.0,
 1656171774.0,
 1656174127.0,
 1656175694.0,
 1656177390.0,
 1656178929.0,
 1656180601.0,
 1656182666.0,
 1656186911.0,
 1656189618.0,
 1656191904.0,
 1656194733.0,
 1656199427.0,
 1656203363.0,
 1656212778.0,
 1656221052.0,
 1656228680.0,
 1656233774.0,
 1656235618.0,
 1656238175.0,
 1656241639.0,
 1656244524.0,
 1656249272.0,
 1656253133.0,
 1656257364.0,
 1656261103.0,
 1656266271.0,
 1656269495.0,
 1656271998.0,
 1656276045.0,
 1656278705.0,
 1656280657.0,
 1656283265.0,
 1656287204.0,
 1656295449.0,
 1656304434.0,
 1656309587.0,
 1656315035.0,
 1656318706.0,
 165632153

In [ ]:
for index, row in calc_df.iterrows():
    if row['Time_to_Fill'] < 0:
        result = 0
        calc_df.at[index, 'Time_to_Fill'] = result

In [43]:
negative_values = calc_df.loc[calc_df['Time_to_Fill'] < 0, 'Time_to_Fill']
negative_values

Series([], Name: Time_to_Fill, dtype: float64)

## Calculate Inflow and Average Inflow

In [45]:
calc_df['Inflow'] = float()
calc_df['Avg_Inflow'] = float()

In [46]:
for index, row in calc_df.iterrows():
    if row['Time_to_Fill'] != 0:
        inflow = row['volume_per_foot'] * row['Elevation_dif'] / row['Time_to_Fill']
        calc_df.at[index, 'Inflow'] = inflow
    else:
        pass

In [47]:
sum_dict ={}
count_dict ={}

for index, row in legend.iterrows():
    result = 0
    tag = row['Lift Station']
    sum_dict[tag] = result
    count_dict[tag] = result
    
for index, row in calc_df.iterrows():
    tag = row['LiftStation']
    sum_dict[tag] += row['Inflow']
    count_dict[tag] += 1
    calc_df.at[index, 'Avg_Inflow'] = sum_dict[tag] / count_dict[tag]


In [50]:
calc_df.head(10)

,LiftStation,PumpTagIndex,PumpStartTime,StartWellLevel,PumpStopTime,StopWellLevel,StartUnix,StopUnix,Time_diff,Elevation_dif,wet_well_diam,volume_per_foot,observed_pumping_flow_rate,avg_flow_rate_pump,average_observed_pumping_flow_rate,Time_to_Fill
0,Buena Vista (19),920,2022-06-24 00:11:33,3.200578,2022-06-24 00:13:32,2.189985,1.656029e+09,1.656030e+09,1.983333,3.3,5,147.0,245,245.000000,245.000000,1970-01-01 00:00:00
1,Buena Vista (19),918,2022-06-24 03:33:31,3.196136,2022-06-24 03:35:31,2.187765,1.656042e+09,1.656042e+09,2.000000,3.3,5,147.0,243,243.000000,244.000000,0 days 03:19:59
2,Buena Vista (19),920,2022-06-24 06:24:22,3.198357,2022-06-24 06:26:18,2.232186,1.656052e+09,1.656052e+09,1.933333,3.3,5,147.0,251,248.000000,246.333333,0 days 02:48:51
3,Buena Vista (19),918,2022-06-24 07:32:38,3.200578,2022-06-24 07:34:38,2.229965,1.656056e+09,1.656056e+09,2.000000,3.3,5,147.0,243,243.000000,245.500000,0 days 01:06:20
4,Buena Vista (19),920,2022-06-24 08:49:01,3.196136,2022-06-24 08:50:59,2.192206,1.656061e+09,1.656061e+09,1.966667,3.3,5,147.0,247,247.666667,245.800000,0 days 01:14:23
5,Buena Vista (19),918,2022-06-24 10:03:51,3.200578,2022-06-24 10:05:56,2.214417,1.656065e+09,1.656065e+09,2.083333,3.3,5,147.0,233,239.666667,243.666667,0 days 01:12:52
6,Buena Vista (19),920,2022-06-24 11:33:07,3.200578,2022-06-24 11:35:04,2.192206,1.656070e+09,1.656071e+09,1.950000,3.3,5,147.0,249,248.000000,244.428571,0 days 01:27:11
7,Buena Vista (19),918,2022-06-24 13:06:47,3.200578,2022-06-24 13:08:52,2.192206,1.656076e+09,1.656076e+09,2.083333,3.3,5,147.0,233,238.000000,243.000000,0 days 01:31:43
8,Buena Vista (19),920,2022-06-24 14:59:57,3.200578,2022-06-24 15:01:55,2.201091,1.656083e+09,1.656083e+09,1.966667,3.3,5,147.0,247,247.800000,243.444444,0 days 01:51:05
9,Buena Vista (19),918,2022-06-24 16:00:31,3.200578,2022-06-24 16:02:37,2.223302,1.656086e+09,1.656087e+09,2.100000,3.3,5,147.0,231,236.600000,242.200000,0 days 00:58:36


# Section 5

In this section, I will calculate the actual pump flow rate in GPM, the average flow rate per pump in GPM, and the average pump flow rate in GPM.

Actual pump flow rate - To calculate the actual pump flow rate for each row in calc_df, I simply take the row's observed pumping flow rate value we calculated earlier, and add the row's inflow value.

Average flow rate per pump - To calculate the average flow rate per pump I add the values in the actual pump flow rate column for each pump, and then divide that sum by the count of values for that pump. So if I iterate over 4 rows of the calc_df, and pump 1 has 3 out of the 4 rows with an actual pump flow rate of 100, 120, and 100, the formula will sum these three rows (320) and divide by the number of rows that were related to pump 1 (3). The average flow rate per pump 1 in row 4 would be 106.

Average pump flow rate - to calculate this value I simply add all the actual pump flow rates for both pumps and divide the sum by the count of rows I have iterated over. For example, if I have iterated over 4 rows, and 2 of the rows are pump 1 and 2 are pump 2, I'll add up the values from both pumps (100,110,100,120) and divide the sum by 4 to calculate the average pump flow rate in GPM of 107.5.

## Calculate Actual Pump Flow Rate (GPM)

In [49]:
calc_df['Actual_Pump_Flow_Rate'] = float()

In [50]:
for index, row in calc_df.iterrows():
    result = round(row['observed_pumping_flow_rate'] + row['Inflow'])
    calc_df.at[index, 'Actual_Pump_Flow_Rate'] = result

## Finally Let's calculate our Average Flow Rate Per Pump and the Average Pump Flow Rate in (GPM)

In [58]:
calc_df['Average_Pump_Flow_Rate_(Individual)'] = int()
calc_df['Average_Pump_Flow_Rate_(All_Pumps)'] = int()

In [55]:
pump_sum_dict={}
pump_count_dict={}

for item in valid_pumps:
    pump_sum_dict[item]=0
    pump_count_dict[item]=0
    

for index, row in calc_df.iterrows():
    tag = row['PumpTagIndex']
    pump_sum_dict[tag] += row['Actual_Pump_Flow_Rate']
    pump_count_dict[tag] += 1
    calc_df.at[index,'Average_Pump_Flow_Rate'] = cum_sum / count

In [ ]:
sum_dict ={}
count_dict ={}

for index, row in legend.iterrows():
    result = 0
    tag = row['Lift Station']
    sum_dict[tag] = result
    count_dict[tag] = result

for index, row in calc_df.iterrows():
    tag = row['LiftStation']
    sum_dict[tag] += row['Actual_Pump_Flow_Rate']
    count_dict[tag] += 1
    calc_df.at[index,'Average_Pump_Flow_Rate_(all)'] = cum_sum / count

In [44]:
#calc_df

## Export the data to CSV

In [39]:
calc_df.to_csv("Model Data for Kelly Lift Station.csv")